# 15.1 Rod cutting
Given a rod of length $n$ inches and a table of prices $p_i$ for $i=1,2,,...,n$, determine the maximum revenue $r_n$ by cutting the rod and selling the pieces.

|length $i$|1|2|3|4|5|6|7|8|9|10|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|length $p_i$|1|5|8|9|10|17|17|20|24|30|

The **general recursive relation** between $r_n$ and its preceding term(s) can be expressed by:

$$
\begin{align}
r_n=\max(p_n,r_{n-1}+r_1,r_{n-2}+r_2,...,r_{n-1}+r_1)
\end{align}
$$

* $p_n$ is the revenue for no cutting at all
* the rest $r_{n-i}+r_i$ terms the revenue of cutting the rod into two pieces of length $n-i$ and $i$ each, then optimally cutting the two pieces further to obtain $r_{n-i}$ and $r_i$ respectively.

This recursive relation can be further shortened to:

$$
\begin{align}
r_n=\underset{1\leq i\leq n}{\max}(p_i,r_{n-i})
\end{align}
$$
    
where $p_n$ is the first piece (not divisible) and $r_{n-i}$ is the reminder (divisible). 

In this formula, an optimal solution embodies the solution to only **one related subproblem** of the reminder, rather than both.
## Recursive top-down implementation
Based on the recursive formular $r_n=\underset{1\leq i\leq n}{\max}(p_i,r_{n-i})$, we have a straightforward, **top-down recursion**.

Notice that:
* The initial value q was set at $-1$ instead of $-\infty$ in the book to reduce running time

In [1]:
import numpy as np
p=np.array([0,1,5,8,9,10,17,17,20,24,30])

In [4]:
# top-down recursion
def rod_cut(n):
    if n==0:
        return 0
    q=-1
    for i in range(1,n+1):
        q=max(q,p[i]+rod_cut(n-i))
    return q
cut_rod(p,5) 

13

### Analysis of the running time of recursion
Although recursion is very intuitive, the amount of work done grows explosively as a function of n.  The recursive tree in *Figure 15.3* shows the recursive calls resulting from `cut_rod(4)`.
<img src="img/fig15.3.png" width="700">
Let $T(n)$ denotes the total number of calls made to `cut_rod(n)`. We have
* $T(n)=0$, which is the initial call at the root
* $T(n)=T(0)+\sum_{j=0}^{n-1}T(j)=1+\sum_{j=0}^{n-1}T(j)$

which imply:
$$
\begin{align}
T(n)=2^n
\end{align}
$$

## Using dynamic programming (DP) for optimal rod cutting
**Dynamic programming** converts `cut_rod`into an efficient algorithm in two ways:
### 1. top-down with memoization
In the "naive" recursion above, we observe that the same subproblems were solved **repeatedly**: as in *Figure 15.3*, the nodes $2,1,0$ occur $2,4,8$ times respectively. If you can **store** these solutions rather than **recompute** it everytime, you can save a dramatic amount of computing time! The approach is called ***top-down with memoization***:
1. We write the procedure recursively, but save the solution in an array ot hash table
2. The procedure firsth checks the memorry if a subproblem has a solution already
3. If so, it returns the saved solution
4. If not, it computes the the solution as in a naive recursion

In [5]:
# dynamic programming, approach 1
# top-down with memoization

def rod_cut_mem(p,n):
    r=np.full(n+1,-1)
    return rod_cut_mem_aux(p,n,r)
def rod_cut_mem_aux(p,n,r):
    if r[n]>=0:
        return r[n]
    if n==0:
        return 0
    q=-1
    for i in range(1,n+1):
        q=max(q,p[i]+rod_cut_mem_aux(p,n-i,r))
    r[n]=q
    return r[n]

rod_cut_mem(p,5)  


13

### 2. Bottom-up (iteration)
**Bottom-up** approach uses the natural ordering of the subproblems: a subproblem of size $i$ is "smaller" than a subproblem of size $j$, if $i<j$. Thus, the procedure solves subproblems of size $j=0,1,...,n$ in that order:
1. Line 3 first creates a new array $r[0...n]$ that saves the results of the subproblems
2. Line 4 initialises $r_0$ as $0$
3. Line 7-10 solves each subproblem of size $j$, where $j=1,2,...,n$
    * it is the same as a naive recursion
    * except that Line 8 directly refers ti $r[j-i]$, which is stored
    * instead of making a recursive call to solve the subproblem of size $j-i$
4. Line 11 saves the $r[j]$ solution to the subproblem of size $j$
5. Finally, Line 12 returns $r[n]$

In [99]:
# dynamic programming approach 2
# bottom-up
def rod_cut_bot(n):
    r=np.full(n+1,-1)
    r[0]=0
    q=-1
    for j in range(1,n+1):
        for i in range(1,j+1):
            q=max(q,p[i]+r[j-i])
        r[i]=q
    return r[n]
%timeit rod_cut_bot(p,10)    

33.9 µs ± 65.5 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### Analysis of DP
The **bottom-up** and **top-down** approaches have the **same** asymptotic running time, which is $\Theta (n^2)$. Compared to $\Theta (2^n)$ of naive recursion, DP is much faster!

### Reconstructing a solution
We can extend the DP approach to record not only the optimal value for each subproblem, but also a *choice* that led to the optimal value. The extended version of `rod_cut_bot` computes for each rod size $j$, not only the maximum revenue $r_j$, but also $s_j$, the optimal size of the first piece to cut off.
It is similar to `rod_cut_bot`, except that:
1. Line 1 creates an array $s[0,...,n]$ that holds the optimal size $i$ 
2. Line 10 updates for $s[j]$ at the moment when $q$ reaches its maximum


In [5]:
def extended_bottom_up_cut_rod(p,n):
    r=np.full(n+1,-1)
    s=np.full(n+1,-1)
    r[0]=0
    q=-1
    for j in range(1,n+1):
        for i in range(1,j+1):
            if q<p[i]+r[j-i]:
                q=p[i]+r[j-i]
                s[j]=i           
        r[i]=q
    return r[n],s[j]
extended_bottom_up_cut_rod(p,10) 

(30, 10)

Furthermore, we can retrieve the length of all the pieces cut by calling `extended_bottom_up_cut_rod` repeatedly inside a `while` loop:

***Notice that the psedocodes given in the textbook are not correct. Please use mine below:***

In [11]:
def print_rod_cut_solution(p,n):    
    while n>0:
        r,s=rod_cut_bot_extended(p,n)
        print (s)
        n=n-s #the piece s is chopped off from p, then next while loop
print_rod_cut_solution(p,7)       

1
6
